In [1]:
!pip install google-generativeai
!pip install pytesseract opencv-python easyocr transformers spacy pillow
!pip install git+https://github.com/lukas-blecher/LaTeX-OCR.git
!pip install tiktoken
!pip install protobuf
!pip install sentencepiece
!pip install blobfile
!pip install ollama-python

  Cloning https://github.com/lukas-blecher/LaTeX-OCR.git to /tmp/pip-req-build-87ntd80g
  Running command git clone --filter=blob:none --quiet https://github.com/lukas-blecher/LaTeX-OCR.git /tmp/pip-req-build-87ntd80g
  Resolved https://github.com/lukas-blecher/LaTeX-OCR.git to commit 5c1ac929bd19a7ecf86d5fb8d94771c8969fcb80
  Installing build dependencies ... one
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached httpx-0.26.0-py3-none-any.whl.metadata (7.6 kB)
Using cached httpx-0.26.0-py3-none-any.whl (75 kB)
  Attempting uninstall: httpx
    Found existing installation: httpx 0.28.1
    Uninstalling httpx-0.28.1:
      Successfully uninstalled httpx-0.28.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ollama 0.4.7 requires httpx<0.29,>=0.27, but you have httpx 0.26.0 which is incompatible.


In [2]:
!pip install llama-index qdrant_client torch transformers
!pip install llama-index-llms-ollama

  Using cached httpx-0.28.1-py3-none-any.whl.metadata (7.1 kB)
Using cached httpx-0.28.1-py3-none-any.whl (73 kB)
  Attempting uninstall: httpx
    Found existing installation: httpx 0.26.0
    Uninstalling httpx-0.26.0:
      Successfully uninstalled httpx-0.26.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ollama-python 0.1.2 requires httpx<0.27.0,>=0.26.0, but you have httpx 0.28.1 which is incompatible.


In [3]:
!pip install transformers torch pillow

In [4]:
!pip install 'accelerate>=0.26.0'

In [1]:
try:
    import accelerate
    print(f"Accelerate version: {accelerate.__version__}")
except ImportError:
    print("Accelerate not found")

Accelerate version: 1.3.0


In [2]:
# Imports necessários
import os
import cv2
import numpy as np
from PIL import Image
import easyocr
import pytesseract
from transformers import pipeline
import google.generativeai as genai
from pathlib import Path
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import base64
import warnings
warnings.filterwarnings('ignore')
import glob
import datetime
import re
# Verifica GPU
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Usando device: {device}")

from transformers import DonutProcessor, VisionEncoderDecoderModel
from PIL import Image
import torch
import re


Usando device: cuda


In [3]:
class OCREngine:
    def __init__(self, use_mt5=True, models=None, gemini_api_key=None):
        self.models = models if models else ['easyocr', 'tesseract', 'donut']
        """
        Inicializa OCREngine com modelos selecionados
        
        Args:
            use_mt5: bool, usar MT5 para pós-processamento
            models: list, modelos a serem usados ['easyocr', 'tesseract', 'pix2tex', 'gemini', 'llama', 'deepseek']
            gemini_api_key: str, chave API do Gemini se selecionado
        """
        
        print("[DEBUG] Iniciando OCREngine...")

        # Configura gerenciamento de memória CUDA
        os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'max_split_size_mb:512'
        torch.cuda.empty_cache()
        
        self.use_mt5 = use_mt5
        self.gemini_api_key = gemini_api_key
        
        # Inicializa modelos OCR tradicionais
        if 'easyocr' in self.models:
            try:
                self.reader = easyocr.Reader(['pt'], gpu=False)  # Força modo CPU
            except Exception as e:
                print(f"[ERRO] Falha ao inicializar EasyOCR: {e}")
                self.reader = None
        
        # Inicializa Pix2Tex
        if 'pix2tex' in self.models:
            try:
                from pix2tex.cli import LatexOCR
                self.pix2tex = LatexOCR()
                print("[DEBUG] Pix2Tex inicializado com sucesso")
            except Exception as e:
                print(f"[ERRO] Falha ao inicializar Pix2Tex: {e}")
                self.pix2tex = None
                
        # Inicializa modelos generativos
        if 'gemini' in self.models and gemini_api_key:
            import google.generativeai as genai
            genai.configure(api_key=gemini_api_key)
            self.gemini = genai.GenerativeModel('gemini-1.5-flash')

        if 'donut' in self.models:
            try:
                print("[DEBUG] Iniciando carregamento do DONUT...")
                print("[DEBUG] Carregando processor...")
                self.processor = DonutProcessor.from_pretrained(
                    "naver-clova-ix/donut-base-finetuned-cord-v2"
                )
                
                print("[DEBUG] Configurando parâmetros do modelo...")
                model_kwargs = {
                    "torch_dtype": torch.float16,
                    "device_map": "auto",
                    "low_cpu_mem_usage": True
                }
                
                print("[DEBUG] Carregando modelo...")
                self.donut_model = VisionEncoderDecoderModel.from_pretrained(
                    "naver-clova-ix/donut-base-finetuned-cord-v2",
                    **model_kwargs
                )
                print("[DEBUG] DONUT inicializado com sucesso")
                
            except Exception as e:
                print(f"[ERRO] Falha ao inicializar DONUT: {e}")
                print("[DEBUG] Tentando carregar com configurações alternativas...")
                try:
                    # Tentativa alternativa com menos otimizações
                    self.donut_model = VisionEncoderDecoderModel.from_pretrained(
                        "naver-clova-ix/donut-base-finetuned-cord-v2",
                        torch_dtype=torch.float16
                    ).to('cuda')
                    print("[DEBUG] DONUT inicializado com configuração alternativa")
                except Exception as e2:
                    print(f"[ERRO] Falha na tentativa alternativa: {e2}")
                    self.donut_model = None
        
        if 'llama' in self.models:
            try:
                self.llama = AutoModelForCausalLM.from_pretrained(
                    "meta-llama/Llama-2-70b-chat-hf",
                    device_map="auto",
                    torch_dtype=torch.float16,
                    load_in_4bit=True
                )
                self.llama_tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-70b-chat-hf")
                print("[DEBUG] LLaMA inicializado com sucesso")
            except Exception as e:
                print(f"[ERRO] Falha ao inicializar LLaMA: {e}")
                self.llama = None
                
        if 'deepseek' in self.models:
            try:
                print("[DEBUG] DeepSeek será inicializado via Ollama quando necessário")
                self.deepseek = None  # Não precisa inicializar aqui
            except Exception as e:
                print(f"[ERRO] Falha ao inicializar DeepSeek: {e}")
                self.deepseek = None
                
        print("[DEBUG] OCREngine inicializado com sucesso")

    def process_with_easyocr(self, image):
        """Processa imagem com EasyOCR"""
        try:
            results = self.reader.readtext(image)
            text = " ".join([result[1] for result in results])
            confidence = np.mean([result[2] for result in results]) if results else 0.0
            return {"text": text, "confidence": confidence, "equations": []}
        except Exception as e:
            print(f"[ERRO] Falha no EasyOCR: {e}")
            return {"text": "", "confidence": 0.0, "equations": []}

    def process_with_tesseract(self, image):
        """Processa imagem com Tesseract"""
        try:
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            text = pytesseract.image_to_string(gray, lang='por')
            return {"text": text, "confidence": None, "equations": []}
        except Exception as e:
            print(f"[ERRO] Falha no Tesseract: {e}")
            return {"text": "", "confidence": None, "equations": []}

    def process_with_pix2tex(self, image):
        """Processa imagem com Pix2Tex"""
        try:
            if self.pix2tex is None:
                print("[ERRO] Pix2Tex não foi inicializado")
                return {"text": "", "confidence": None, "equations": []}
            
            # Converte a imagem para PIL Image se necessário
            if isinstance(image, np.ndarray):
                image = Image.fromarray(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    
            latex_result = self.pix2tex(image)
            return {
                "text": "",  # Pix2Tex foca apenas em equações
                "confidence": None,
                "equations": [latex_result] if latex_result else []
            }
        except Exception as e:
            print(f"[ERRO] Falha no Pix2Tex: {e}")
            return {"text": "", "confidence": None, "equations": []}
            
    def process_with_gemini(self, image):
        """Processa imagem com Gemini"""
        try:
            if isinstance(image, np.ndarray):
                image = Image.fromarray(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
                
            prompt = """Analise esta imagem e extraia:
            1. Texto completo
            2. Equações matemáticas em LaTeX
            3. Verifique especialmente funções trigonométricas como sin, cos, tan
            
            Formato LaTeX:
            - Use \sin, \cos, \tan com espaço após
            - Mantenha parênteses e colchetes balanceados
            - Preserve subscritos e superscritos"""
            
            response = self.gemini.generate_content([prompt, image])
            return {"text": response.text, "confidence": None, "equations": []}
        except Exception as e:
            print(f"[ERRO] Falha no Gemini: {e}")
            return {"text": "", "confidence": None, "equations": []}
            
    def process_with_llama(self, image):
        """Processa imagem com LLaMA"""
        try:
            if self.llama is None:
                return {"text": "", "confidence": None}
                
            # Implementar processamento específico do LLaMA
            return {"text": "LLaMA processing not implemented", "confidence": None}
        except Exception as e:
            print(f"[ERRO] Falha no LLaMA: {e}")
            return {"text": "", "confidence": None}

    def process_with_deepseek(self, image):
        """Processa imagem com DeepSeek via llama-index e Ollama"""
        try:
            import torch
            from llama_index.llms.ollama import Ollama  # Import corrigido
            from PIL import Image
            import io
            import base64
            
            # Configurações de memória CUDA
            torch.cuda.empty_cache()
            torch.cuda.set_per_process_memory_fraction(0.8)
            
            print("[DEBUG] Iniciando processamento com DeepSeek...")
            print("[DEBUG] VRAM configurada para 80% do total disponível")
            
            # Configura o Ollama
            llm = Ollama(model="deepseek-r1:32b", temperature=0)
            
            # Prepara a imagem
            if isinstance(image, np.ndarray):
                image = Image.fromarray(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
            
            buffered = io.BytesIO()
            image.save(buffered, format="PNG")
            img_str = base64.b64encode(buffered.getvalue()).decode()
            
            # Prompt para análise
            prompt = """Analise esta imagem e forneça:
            1. O texto completo presente
            2. Equações matemáticas em LaTeX
            3. Mantenha a formatação original das funções trigonométricas
            
            Use $$ para equações em display mode e $ para equações inline."""
            
            # Realiza a chamada
            response = llm.complete(prompt)
            
            # Processa o resultado
            text = response.text
            
            # Extrai equações
            import re
            equations = []
            patterns = [r'\$\$(.*?)\$\$', r'\$(.*?)\$']
            
            for pattern in patterns:
                equations.extend(re.findall(pattern, text, re.DOTALL))
            
            return {
                "text": text,
                "confidence": None,
                "equations": equations
            }
                
        except Exception as e:
            print(f"[ERRO] Falha no DeepSeek: {e}")
            import traceback
            print(f"[ERRO] Detalhes:\n{traceback.format_exc()}")
            return {"text": "", "confidence": None, "equations": []}
    
    def process_with_donut(self, image):
        """Processa imagem usando DONUT via Hugging Face"""
        try:
            if self.donut_model is None:
                print("[ERRO] Modelo DONUT não foi inicializado corretamente")
                return {"text": "", "confidence": None, "equations": []}
                
            print("[DEBUG] Iniciando processamento DONUT...")
            
            # Prepara a imagem
            if isinstance(image, np.ndarray):
                print("[DEBUG] Convertendo imagem para PIL...")
                image = Image.fromarray(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
                
            print("[DEBUG] Processando imagem com processor...")
            pixel_values = self.processor(image, return_tensors="pt").pixel_values
            
            print("[DEBUG] Movendo tensors para GPU...")
            pixel_values = pixel_values.to(device="cuda", dtype=torch.float16)
            
            print("[DEBUG] Gerando output...")
            with torch.cuda.amp.autocast():
                outputs = self.donut_model.generate(
                    pixel_values,
                    max_length=512,
                    num_beams=4,
                    early_stopping=True
                )
                
            print("[DEBUG] Decodificando resultado...")
            sequence = self.processor.batch_decode(outputs)[0]
            sequence = sequence.replace("<s>", "").replace("</s>", "")
            
            # Extrai equações se presentes
            equations = re.findall(r'\$(.*?)\$', sequence)
            
            print("[DEBUG] Processamento DONUT concluído com sucesso")
            return {
                "text": sequence,
                "confidence": None,
                "equations": equations
            }
            
        except Exception as e:
            print(f"[ERRO] Falha no processamento DONUT: {e}")
            import traceback
            print(f"[ERRO] Detalhes:\n{traceback.format_exc()}")
            return {"text": "", "confidence": None, "equations": []}
    
    def generate_accessible_html(self, image_path, results):
        """Gera HTML acessível com os resultados de todos os modelos"""
        
        with open(image_path, "rb") as img_file:
            img_base64 = base64.b64encode(img_file.read()).decode()
        
        html = f"""
        <!DOCTYPE html>
        <html lang="pt-BR">
        <head>
            <meta charset="UTF-8">
            <meta name="viewport" content="width=device-width, initial-scale=1.0">
            <title>Análise Multimodal OCR</title>
            
            <!-- MathJax Config -->
            <script>
            MathJax = {{
                tex: {{
                    inlineMath: [['$', '$'], ['\\\\(', '\\\\)']],
                    displayMath: [['$$', '$$'], ['\\\\[', '\\\\]']],
                    processEscapes: true,
                    packages: ['base', 'ams', 'noerrors', 'noundefined']
                }},
                svg: {{
                    fontCache: 'global'
                }},
                options: {{
                    renderActions: {{
                        addMenu: [0, '', ''],
                        checkLoading: [200, '', '']
                    }}
                }}
            }};
            </script>
            <script id="MathJax-script" async
                    src="https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-svg.js">
            </script>
            
            <style>
                :root {{
                    --text-color: #333;
                    --bg-color: #fff;
                    --secondary-bg: #f8f9fa;
                    --accent-color: #3498db;
                    --error-color: #e74c3c;
                    --success-color: #2ecc71;
                }}
                
                body {{
                    font-family: system-ui, -apple-system, sans-serif;
                    line-height: 1.6;
                    color: var(--text-color);
                    background: var(--bg-color);
                    margin: 0;
                    padding: 20px;
                    transition: all 0.3s ease;
                }}
                
                .container {{
                    max-width: 1200px;
                    margin: 0 auto;
                }}
                
                .image-section {{
                    text-align: center;
                    margin: 2em 0;
                    padding: 1em;
                    background: var(--secondary-bg);
                    border-radius: 8px;
                }}
                
                .image-section img {{
                    max-width: 100%;
                    height: auto;
                    border-radius: 4px;
                    box-shadow: 0 2px 4px rgba(0,0,0,0.1);
                }}
                
                .results-grid {{
                    display: grid;
                    grid-template-columns: repeat(auto-fit, minmax(350px, 1fr));
                    gap: 2em;
                    margin: 2em 0;
                }}
                
                .model-result {{
                    background: var(--secondary-bg);
                    padding: 1.5em;
                    border-radius: 8px;
                    box-shadow: 0 2px 4px rgba(0,0,0,0.1);
                }}
                
                .model-result h2 {{
                    color: var(--accent-color);
                    margin-top: 0;
                    border-bottom: 2px solid var(--accent-color);
                    padding-bottom: 0.5em;
                }}
                
                .equations-section {{
                    margin: 1em 0;
                    padding: 1em;
                    background: rgba(255,255,255,0.5);
                    border-radius: 4px;
                }}
                
                .equation {{
                    margin: 1em 0;
                    padding: 0.5em;
                    background: white;
                    border-radius: 4px;
                    overflow-x: auto;
                }}
                
                .equation-trig {{
                    border-left: 3px solid var(--accent-color);
                }}
                
                .confidence {{
                    display: inline-block;
                    padding: 0.3em 0.6em;
                    border-radius: 4px;
                    background: var(--accent-color);
                    color: white;
                    font-size: 0.9em;
                }}
                
                .error-message {{
                    color: var(--error-color);
                    padding: 0.5em;
                    margin: 0.5em 0;
                    border-left: 3px solid var(--error-color);
                }}
                
                .accessibility-controls {{
                    position: fixed;
                    bottom: 20px;
                    right: 20px;
                    background: var(--bg-color);
                    padding: 1em;
                    border-radius: 8px;
                    box-shadow: 0 2px 10px rgba(0,0,0,0.1);
                    display: flex;
                    gap: 0.5em;
                    flex-wrap: wrap;
                    max-width: 300px;
                    z-index: 1000;
                }}
                
                button {{
                    padding: 0.5em 1em;
                    border: none;
                    border-radius: 4px;
                    background: var(--accent-color);
                    color: white;
                    cursor: pointer;
                    transition: all 0.2s ease;
                }}
                
                button:hover {{
                    transform: translateY(-2px);
                    box-shadow: 0 2px 4px rgba(0,0,0,0.2);
                }}
                
                button:focus {{
                    outline: 3px solid rgba(52, 152, 219, 0.5);
                    outline-offset: 2px;
                }}
                
                /* Dark mode */
                body.dark-mode {{
                    --text-color: #e0e0e0;
                    --bg-color: #1a1a1a;
                    --secondary-bg: #2d2d2d;
                }}
                
                /* High contrast */
                body.high-contrast {{
                    --text-color: #fff;
                    --bg-color: #000;
                    --secondary-bg: #333;
                    --accent-color: #fff;
                }}
                
                @media (prefers-reduced-motion: reduce) {{
                    * {{
                        transition: none !important;
                    }}
                }}
                
                @media (max-width: 768px) {{
                    .results-grid {{
                        grid-template-columns: 1fr;
                    }}
                    
                    .accessibility-controls {{
                        left: 20px;
                        right: 20px;
                        max-width: unset;
                    }}
                }}
                
                /* Print styles */
                @media print {{
                    .accessibility-controls {{
                        display: none;
                    }}
                    
                    body {{
                        background: white;
                    }}
                    
                    .model-result {{
                        break-inside: avoid;
                        page-break-inside: avoid;
                    }}
                }}
            </style>
        </head>
        <body>
            <div class="container">
                <header>
                    <h1>Resultados da Análise OCR</h1>
                </header>
                
                <main>
                    <section class="image-section" aria-label="Imagem analisada">
                        <img src="data:image/jpeg;base64,{img_base64}" 
                             alt="Imagem original analisada" 
                             role="img">
                    </section>
                    
                    <div class="results-grid">
        """
        
        for model_name, result in results.items():
            html += f"""
                        <section class="model-result" role="region" aria-label="Resultado {model_name}">
                            <h2>{model_name.upper()}</h2>
                            
                            <div class="text-section">
                                <h3>Texto Detectado</h3>
                                <div role="text">{result['text']}</div>
                            </div>
                            
                            <div class="equations-section">
                                <h3>Equações Detectadas</h3>
            """
            
            if result.get('equations'):
                for i, eq in enumerate(result['equations'], 1):
                    # Verifica se é uma equação trigonométrica
                    eq_class = "equation-trig" if any(trig in eq for trig in ['\\sin', '\\cos', '\\tan']) else ""
                    html += f"""
                                <div class="equation {eq_class}" role="math" aria-label="Equação {i}">
                                    $${eq}$$
                                </div>
                    """
            else:
                html += """
                                <p>Nenhuma equação detectada</p>
                """
            
            if result.get('confidence') is not None:
                confidence = f"{result['confidence']:.2%}" if isinstance(result['confidence'], float) else result['confidence']
                html += f"""
                            <div class="confidence">
                                Confiança: {confidence}
                            </div>
                """
            
            html += """
                        </section>
            """
        
        html += """
                    </div>
                </main>
                
                <div class="accessibility-controls" role="complementary">
                    <button onclick="toggleDarkMode()" aria-label="Alternar modo escuro">
                        🌓 Modo Escuro
                    </button>
                    <button onclick="toggleHighContrast()" aria-label="Alternar alto contraste">
                        👁️ Alto Contraste
                    </button>
                    <button onclick="toggleSpeech()" aria-label="Ler conteúdo" id="speechButton">
                        🔊 Ler
                    </button>
                    <button onclick="increaseFontSize()" aria-label="Aumentar fonte">
                        A+
                    </button>
                    <button onclick="decreaseFontSize()" aria-label="Diminuir fonte">
                        A-
                    </button>
                </div>
            </div>
            
            <script>
                let isReading = false;
                const speechUtterance = new SpeechSynthesisUtterance();
                speechUtterance.lang = 'pt-BR';
                
                function toggleDarkMode() {
                    document.body.classList.toggle('dark-mode');
                    document.body.classList.remove('high-contrast');
                    reloadMathJax();
                }
                
                function toggleHighContrast() {
                    document.body.classList.toggle('high-contrast');
                    document.body.classList.remove('dark-mode');
                    reloadMathJax();
                }
                
                function toggleSpeech() {
                    if (isReading) {
                        window.speechSynthesis.cancel();
                        isReading = false;
                        document.getElementById('speechButton').innerHTML = '🔊 Ler';
                    } else {
                        const textElements = document.querySelectorAll('[role="text"]');
                        let fullText = '';
                        textElements.forEach(el => {
                            fullText += el.textContent + ' ';
                        });
                        
                        speechUtterance.text = fullText;
                        speechUtterance.onend = function() {
                            isReading = false;
                            document.getElementById('speechButton').innerHTML = '🔊 Ler';
                        };
                        
                        window.speechSynthesis.speak(speechUtterance);
                        isReading = true;
                        document.getElementById('speechButton').innerHTML = '🔊 Parar';
                    }
                }
                
                let currentFontSize = 16;
                function increaseFontSize() {
                    currentFontSize = Math.min(currentFontSize * 1.1, 24);
                    document.body.style.fontSize = currentFontSize + 'px';
                    reloadMathJax();
                }
                
                function decreaseFontSize() {
                    currentFontSize = Math.max(currentFontSize * 0.9, 12);
                    document.body.style.fontSize = currentFontSize + 'px';
                    reloadMathJax();
                }
                
                function reloadMathJax() {
                    if (typeof MathJax !== 'undefined') {
                        MathJax.typesetClear();
                        MathJax.typesetPromise();
                    }
                }
                
                document.addEventListener('DOMContentLoaded', function() {
                    MathJax.startup.promise.then(() => {
                        console.log('MathJax inicializado com sucesso');
                    }).catch(err => {
                        console.error('Erro ao inicializar MathJax:', err);
                    });
                });
                
                // Keyboard shortcuts
                document.addEventListener('keydown', (e) => {
                    if (e.altKey) {
                        switch(e.key) {
                            case 'd': toggleDarkMode(); break;
                            case 'c': toggleHighContrast(); break;
                            case 's': toggleSpeech(); break;
                            case '+': increaseFontSize(); break;
                            case '-': decreaseFontSize(); break;
                        }
                    }
                });
            </script>
        </body>
        </html>
        """
        
        return html
    
    def generate_gemini_text_html(self, gemini_result):
        """Gera HTML para visualização do texto e equações do Gemini com recursos de acessibilidade"""
        
        # Processa o texto para equações LaTeX
        text = gemini_result['text']
        text = re.sub(r'\$([^$]+)\$', r'\\(\1\\)', text)
        text = re.sub(r'\$\$([^$]+)\$\$', r'\[\1\]', text)
        
        html = f"""
        <!DOCTYPE html>
        <html lang="pt-BR">
        <head>
            <meta charset="UTF-8">
            <meta name="viewport" content="width=device-width, initial-scale=1.0">
            <title>Texto e Equações - Gemini</title>
            
            <!-- MathJax Config -->
            <script>
            MathJax = {{
                tex: {{
                    inlineMath: [['\\\\(', '\\\\)']],
                    displayMath: [['\\\\[', '\\\\]']]
                }},
                svg: {{
                    fontCache: 'global'
                }}
            }};
            </script>
            <script type="text/javascript" id="MathJax-script" async
                    src="https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-mml-chtml.js">
            </script>
            
            <style>
                :root {{
                    --text-color: #333;
                    --bg-color: #fff;
                    --secondary-bg: #f8f9fa;
                }}
                
                /* Base styles */
                body {{
                    font-family: system-ui, -apple-system, sans-serif;
                    line-height: 1.6;
                    max-width: 800px;
                    margin: 0 auto;
                    padding: 20px;
                    color: var(--text-color);
                    background: var(--bg-color);
                    transition: all 0.3s ease;
                }}
                
                /* Content container */
                .content {{
                    padding: 20px;
                    background: var(--secondary-bg);
                    border-radius: 10px;
                    margin: 20px 0;
                    box-shadow: 0 2px 5px rgba(0,0,0,0.1);
                }}
                
                /* Math display */
                .math-display {{
                    overflow-x: auto;
                    padding: 1em;
                    margin: 1em 0;
                    background: var(--bg-color);
                    border-radius: 5px;
                }}
                
                /* Dark mode */
                body.dark-mode {{
                    --text-color: #e0e0e0;
                    --bg-color: #1a1a1a;
                    --secondary-bg: #2d2d2d;
                }}
                
                /* High contrast */
                body.high-contrast {{
                    --text-color: #fff;
                    --bg-color: #000;
                    --secondary-bg: #222;
                }}
                
                /* Accessibility controls */
                .accessibility-controls {{
                    position: fixed;
                    bottom: 20px;
                    right: 20px;
                    background: var(--secondary-bg);
                    padding: 15px;
                    border-radius: 8px;
                    box-shadow: 0 2px 10px rgba(0,0,0,0.1);
                    display: flex;
                    flex-wrap: wrap;
                    gap: 10px;
                    z-index: 1000;
                    max-width: 300px;
                }}
                
                /* Buttons */
                button {{
                    padding: 8px 12px;
                    border: none;
                    border-radius: 4px;
                    background: #3498db;
                    color: white;
                    cursor: pointer;
                    transition: all 0.3s ease;
                    font-size: 14px;
                    display: flex;
                    align-items: center;
                    gap: 5px;
                }}
                
                button:hover {{
                    background: #2980b9;
                    transform: translateY(-2px);
                }}
                
                button:focus {{
                    outline: 3px solid #74b9ff;
                    outline-offset: 2px;
                }}
                
                /* Responsive design */
                @media (max-width: 768px) {{
                    .accessibility-controls {{
                        left: 20px;
                        right: 20px;
                        max-width: unset;
                    }}
                }}
            </style>
        </head>
        <body>
            <main>
                <h1>Interpretação do Texto</h1>
                
                <div class="content" role="region" aria-label="Texto interpretado">
                    <div id="text-content">
                        {text}
                    </div>
                </div>
                
                <div class="accessibility-controls" role="complementary">
                    <button onclick="toggleDarkMode()" aria-label="Alternar modo escuro">
                        🌓 Modo Escuro</button>
                    <button onclick="toggleHighContrast()" aria-label="Alternar alto contraste">
                        👁️ Alto Contraste</button>
                    <button onclick="toggleSpeech()" aria-label="Ler texto" id="speechButton">
                        🔊 Ler</button>
                    <button onclick="increaseFontSize()" aria-label="Aumentar fonte">
                        A+</button>
                    <button onclick="decreaseFontSize()" aria-label="Diminuir fonte">
                        A-</button>
                </div>
            </main>
            
            <script>
                let isReading = false;
                const speechUtterance = new SpeechSynthesisUtterance();
                speechUtterance.lang = 'pt-BR';
                
                // Dark mode toggle
                function toggleDarkMode() {{
                    document.body.classList.toggle('dark-mode');
                    document.body.classList.remove('high-contrast');
                    reloadMathJax();
                }}
                
                // High contrast toggle
                function toggleHighContrast() {{
                    document.body.classList.toggle('high-contrast');
                    document.body.classList.remove('dark-mode');
                    reloadMathJax();
                }}
                
                // Speech synthesis
                function toggleSpeech() {{
                    if (isReading) {{
                        window.speechSynthesis.cancel();
                        isReading = false;
                        document.getElementById('speechButton').innerHTML = '🔊 Ler';
                    }} else {{
                        const text = document.getElementById('text-content').textContent;
                        speechUtterance.text = text;
                        
                        speechUtterance.onend = function() {{
                            isReading = false;
                            document.getElementById('speechButton').innerHTML = '🔊 Ler';
                        }};
                        
                        window.speechSynthesis.speak(speechUtterance);
                        isReading = true;
                        document.getElementById('speechButton').innerHTML = '🔊 Parar';
                    }}
                }}
                
                // Font size controls
                let currentFontSize = parseFloat(getComputedStyle(document.body).fontSize);
                
                function increaseFontSize() {{
                    currentFontSize = Math.min(currentFontSize * 1.1, 32);
                    document.body.style.fontSize = currentFontSize + 'px';
                    reloadMathJax();
                }}
                
                function decreaseFontSize() {{
                    currentFontSize = Math.max(currentFontSize * 0.9, 12);
                    document.body.style.fontSize = currentFontSize + 'px';
                    reloadMathJax();
                }}
                
                // MathJax reload
                function reloadMathJax() {{
                    if (typeof MathJax !== 'undefined') {{
                        MathJax.typesetClear();
                        MathJax.typesetPromise();
                    }}
                }}
                
                // Keyboard shortcuts
                document.addEventListener('keydown', (e) => {{
                    if (e.altKey) {{
                        switch(e.key) {{
                            case 'd': toggleDarkMode(); break;
                            case 'c': toggleHighContrast(); break;
                            case 's': toggleSpeech(); break;
                            case '+': increaseFontSize(); break;
                            case '-': decreaseFontSize(); break;
                        }}
                    }}
                }});
            </script>
        </body>
        </html>
        """
        
        # Salva o HTML
        output_dir = 'resultados/gemini'
        os.makedirs(output_dir, exist_ok=True)
        timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
        html_path = os.path.join(output_dir, f'gemini_texto_{timestamp}.html')
        
        with open(html_path, 'w', encoding='utf-8') as f:
            f.write(html)
            
        return html_path
    
    def process_image(self, image_path):
        """Processa imagem com todos os modelos selecionados"""
        print(f"\n[DEBUG] Processando {image_path}")
        
        try:
            image = cv2.imread(image_path)
            if image is None:
                raise ValueError("Não foi possível carregar a imagem")
                
            results = {}
            
            # Processa com cada modelo selecionado
            if 'easyocr' in self.models:
                results['easyocr'] = self.process_with_easyocr(image)
                
            if 'tesseract' in self.models:
                results['tesseract'] = self.process_with_tesseract(image)
                
            if 'pix2tex' in self.models:
                results['pix2tex'] = self.process_with_pix2tex(image)
                
            if 'gemini' in self.models:
                results['gemini'] = self.process_with_gemini(image)
                
            if 'llama' in self.models:
                results['llama'] = self.process_with_llama(image)
                
            if 'deepseek' in self.models:
                results['deepseek'] = self.process_with_deepseek(image)
                
            # Gera HTML com resultados
            html = self.generate_accessible_html(image_path, results)
            
            # Salva HTML
            output_dir = 'resultados'
            os.makedirs(output_dir, exist_ok=True)
            html_path = os.path.join(output_dir, f'resultado_{Path(image_path).stem}.html')
            
            with open(html_path, 'w', encoding='utf-8') as f:
                f.write(html)

            # Dentro do método process_image, após processar com Gemini
            if 'gemini' in self.models and 'gemini' in results:
                gemini_text_path = self.generate_gemini_text_html(results['gemini'])
                print(f"Resultado texto Gemini salvo em: {gemini_text_path}")
    
            return {
                'results': results,
                'html_path': html_path
            }
            
        except Exception as e:
            print(f"[ERRO] Falha no processamento: {e}")
            import traceback
            print(f"[ERRO] Detalhes: {traceback.format_exc()}")
            return None

In [8]:
def main():
    """Interface interativa para processamento de imagens"""
    # Configurações básicas
    pasta_entrada = 'assets/semana_07ex/'

    # Debug: Check if input directory exists
    print(f"\nDebug: Verificando pasta de entrada: {pasta_entrada}")
    if os.path.exists(pasta_entrada):
        print(f"✓ Pasta encontrada")
        print("Arquivos encontrados:")
        for file in os.listdir(pasta_entrada):
            print(f"  - {file}")
    else:
        print(f"✗ Pasta não encontrada: {pasta_entrada}")
        return None

    # Lista métodos disponíveis
    metodos_disponiveis = {
        '1': ('easyocr', 'OCR usando EasyOCR (robusto para múltiplos idiomas)'),
        '2': ('tesseract', 'OCR usando Tesseract (básico)'),
        '3': ('pix2tex', 'OCR especializado em equações matemáticas'),
        '4': ('gemini', 'Modelo Gemini 1.5 Flash (requer API key)'),
        '5': ('donut', 'DONUT via Hugging Face (otimizado para documentos)'),
    }

    # Mostra opções disponíveis
    print("\n=== Métodos de OCR Disponíveis ===")
    for key, (metodo, desc) in metodos_disponiveis.items():
        print(f"{key}: {desc}")

    # Seleciona métodos
    while True:
        try:
            escolha = input("\nEscolha os números dos métodos (separados por espaço): ").strip()
            escolhas = escolha.split()
            metodos_escolhidos = [metodos_disponiveis[e][0] for e in escolhas]
            break
        except KeyError:
            print("Erro: Escolha apenas números válidos da lista acima.")

    # Se Gemini foi selecionado, solicita API key
    #gemini_api_key = None
    gemini_api_key = "" #ei sai daqui
    if 'gemini' in metodos_escolhidos:
        gemini_api_key = input("\nDigite sua API key do Gemini: ").strip()
        if not gemini_api_key:
            print("API key não fornecida. Removendo Gemini dos métodos.")
            metodos_escolhidos.remove('gemini')

    # Seleção de páginas
    print("\n=== Opções de Processamento ===")
    print("1: Todas as páginas")
    print("2: Página específica")
    print("3: Intervalo de páginas")

    while True:
        opcao_paginas = input("\nEscolha uma opção: ")
        try:
            if opcao_paginas == '1':
                pattern = os.path.join(pasta_entrada, '*.png')
                arquivos = sorted(glob.glob(pattern))
                pages = list(range(1, len(arquivos) + 1))
                break
            elif opcao_paginas == '2':
                num_pagina = int(input("Digite o número da página: "))
                pages = [num_pagina]
                break
            elif opcao_paginas == '3':
                inicio = int(input("Página inicial: "))
                fim = int(input("Página final: "))
                if inicio > fim:
                    print("Erro: Página inicial deve ser menor que a final")
                    continue
                pages = list(range(inicio, fim + 1))
                break
            else:
                print("Opção inválida. Escolha 1, 2 ou 3.")
        except ValueError:
            print("Por favor, digite números válidos.")

    # Inicializa OCREngine
    print("\nInicializando OCR Engine...")
    engine = OCREngine(
        use_mt5=True,
        models=metodos_escolhidos,
        gemini_api_key=gemini_api_key
    )

    # Processamento
    results = {}
    total_pages = len(pages)

    print(f"\nIniciando processamento de {total_pages} página(s)...")
    print(f"Métodos selecionados: {', '.join(metodos_escolhidos)}")

    for idx, page_num in enumerate(pages, 1):
        print(f"\nProcessando página {page_num} ({idx}/{total_pages})...")
        image_path = os.path.join(pasta_entrada, f'pagina_{page_num}.png')

        if not os.path.exists(image_path):
            print(f"Debug: Procurando arquivos alternativos com padrão: *_{page_num}.png")
            arquivos = glob.glob(os.path.join(pasta_entrada, f'*_{page_num}.png'))
            if arquivos:
                image_path = arquivos[0]
                print(f"Debug: Arquivo alternativo encontrado: {image_path}")
            else:
                print(f"Debug: Nenhum arquivo encontrado para página {page_num}")
                continue

        try:
            result = engine.process_image(image_path)
            if result:
                results[page_num] = result
                print(f"✓ Página {page_num} processada com sucesso")
                print(f"  Resultados salvos em: {result['html_path']}")
            else:
                print(f"✗ Falha no processamento da página {page_num}")

        except Exception as e:
            print(f"Erro ao processar página {page_num}: {str(e)}")
            import traceback
            print("Detalhes do erro:", traceback.format_exc())

    print("\n=== Processamento Concluído ===")
    print(f"Total de páginas processadas: {len(results)}/{total_pages}")
    
    return results

if __name__ == "__main__":
    results = main()


Debug: Verificando pasta de entrada: assets/semana_07ex/
✓ Pasta encontrada
Arquivos encontrados:
  - pagina_1_acessivel.html
  - pagina_1_daces.html
  - pagina_1_daces-1.html
  - pagina_4.png
  - pagina_5.png
  - pagina_1.png
  - pagina_1_daces-2.html
  - pagina_1_tecnico.html
  - .ipynb_checkpoints
  - pagina_2.png
  - pagina_3.png
  - pagina_4.jpg

=== Métodos de OCR Disponíveis ===
1: OCR usando EasyOCR (robusto para múltiplos idiomas)
2: OCR usando Tesseract (básico)
3: OCR especializado em equações matemáticas
4: Modelo Gemini 1.5 Flash (requer API key)
5: DONUT via Hugging Face (otimizado para documentos)



Escolha os números dos métodos (separados por espaço):  1 2



=== Opções de Processamento ===
1: Todas as páginas
2: Página específica
3: Intervalo de páginas



Escolha uma opção:  2
Digite o número da página:  1


Using CPU. Note: This module is much faster with a GPU.



Inicializando OCR Engine...
[DEBUG] Iniciando OCREngine...
[DEBUG] OCREngine inicializado com sucesso

Iniciando processamento de 1 página(s)...
Métodos selecionados: easyocr, tesseract

Processando página 1 (1/1)...

[DEBUG] Processando assets/semana_07ex/pagina_1.png
[ERRO] Falha no EasyOCR: CUDA error: CUDA-capable device(s) is/are busy or unavailable
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.

✓ Página 1 processada com sucesso
  Resultados salvos em: resultados/resultado_pagina_1.html

=== Processamento Concluído ===
Total de páginas processadas: 1/1


In [5]:
import re

class GeminiOCR:
    def __init__(self, api_key):
        """
        Inicializa o GeminiOCR com a chave API do Gemini
        """
        self.api_key = api_key
        import google.generativeai as genai
        genai.configure(api_key=api_key)
        self.model = genai.GenerativeModel('gemini-1.5-flash')

    def process_image(self, image_path):
        """
        Interface principal para processamento da imagem
        """
        try:
            print("\nEscolha o método de processamento:")
            print("1: HTML Simplificado (sem LaTeX, texto por extenso)")
            print("2: HTML Técnico (com LaTeX e recursos de acessibilidade)")
            
            while True:
                choice = input("\nDigite sua escolha (1 ou 2): ").strip()
                if choice in ['1', '2']:
                    break
                print("Opção inválida. Digite 1 ou 2.")

            from PIL import Image
            image = Image.open(image_path)

            if choice == '1':
                return self._process_simple(image_path, image)
            else:
                return self._process_technical(image_path, image)

        except Exception as e:
            print(f"Erro no processamento: {e}")
            return None

    def _process_simple(self, image_path, image):
        """Processa no modo simplificado"""
        prompt = """
        Descreva diretamente o conteúdo desta lista de exercícios de forma natural e acessível para um leitor de tela.
        Importante:
        - Não mencione que é uma descrição
        - Não use frases como "a imagem mostra" ou "aqui está"
        - Escreva como se estivesse lendo o conteúdo diretamente
        - Use linguagem clara e direta
        - Estruture o texto em seções lógicas
        - Para equações matemáticas, use palavras por extenso
        - Indique parênteses verbalmente apenas em equações
        """
        
        result = self.model.generate_content([prompt, image])
        
        html = f"""
        <!DOCTYPE html>
        <html lang="pt-BR">
        <head>
            <meta charset="UTF-8">
            <meta name="viewport" content="width=device-width, initial-scale=1.0">
            <title>Descrição Textual</title>
            <style>
                body {{
                    font-family: Arial, sans-serif;
                    line-height: 1.6;
                    max-width: 800px;
                    margin: 0 auto;
                    padding: 20px;
                }}
                .header {{
                    background-color: #f0f0f0;
                    padding: 15px;
                    border-radius: 5px;
                    margin-bottom: 20px;
                }}
                .section {{
                    background-color: #ffffff;
                    padding: 15px;
                    margin: 15px 0;
                    border-left: 4px solid #2196F3;
                    border-radius: 0 5px 5px 0;
                }}
                h1, h2, h3 {{
                    color: #333;
                    margin-top: 20px;
                }}
                ul, ol {{
                    padding-left: 20px;
                }}
                li {{
                    margin: 8px 0;
                }}
                .footer {{
                    margin-top: 20px;
                    padding: 10px;
                    background-color: #f0f0f0;
                    border-radius: 5px;
                    font-style: italic;
                }}
            </style>
        </head>
        <body>
            <div class="content">
                {result.text}
            </div>
        </body>
        </html>
        """
        
        # Formata o texto para melhor estruturação
        formatted_text = result.text.replace('\n\n', '</div><div class="section">')
        formatted_text = f'<div class="section">{formatted_text}</div>'
        
        output_path = f"{image_path.rsplit('.', 1)[0]}_simples.html"
        with open(output_path, 'w', encoding='utf-8') as f:
            f.write(html)
        return output_path


    def _clean_text(self, response):
        """Limpa o texto removendo artefatos indesejados"""
        # Primeiro extraímos o texto do objeto GenerateContentResponse
        if hasattr(response, 'text'):
            text = response.text
        else:
            raise ValueError("Resposta do Gemini não contém texto")
            
        replacements = {
            r'\u00e7': 'ç',
            r'\u00f5': 'õ',
            r'\u00e3': 'ã',
            r'\n': '\n',
            r'\t': '    ',
            '\u2013': '-',
            '\u2014': '-',
            r'\u00ed': 'í',
            r'\u00e1': 'á',
            r'\u00e9': 'é',
            r'\u00fa': 'ú',
            r'\u00f3': 'ó',
            '\\\\': '\\',
            '\u20134': '-',
            '\u22121': '-',
            '\u20132': '-',
            '\u20133': '-',
        }
        
        # Remove sequências de escape e códigos Unicode
        cleaned_text = text
        for old, new in replacements.items():
            cleaned_text = cleaned_text.replace(old, new)
        
        # Remove caracteres de controle remanescentes
        cleaned_text = re.sub(r'\\u[0-9a-fA-F]{4}', '', cleaned_text)
        
        # Remove espaços múltiplos
        cleaned_text = re.sub(r'\s+', ' ', cleaned_text)
        
        # Garante quebras de linha apropriadas
        cleaned_text = re.sub(r'\\n', '\n', cleaned_text)
        
        return cleaned_text

    def _process_technical(self, image_path, image):
        """Processa no modo técnico"""
        prompt = r"""
        Analise esta imagem e extraia:
        1. Texto completo preservando a formatação
        2. Equações matemáticas em LaTeX (use $$ para display mode, $ para inline)
        3. Mantenha a precisão técnica do conteúdo
        4. Preserve símbolos especiais e notações matemáticas
        
        IMPORTANTE para notação matemática:
        1. Use \arcsin (não extarcsen ou arcsen)
        2. Use \arccos (não extarccos ou arccos)
        3. Use \arctan (não extarctan ou arctan)
        4. Use \sin (não sen)
        5. Use \cos (não cos)
        6. Use \tan (não tan)
        7. Use \sqrt{} para raiz quadrada
        8. Use \frac{}{} para frações
        
        Para expressões matemáticas:
        - Use $ $ para equações inline
        - Use $$ $$ para equações em display mode
        - Preserve todos os parênteses e sinais
        - Mantenha a formatação precisa das funções trigonométricas
        
        Mantenha a estrutura do documento e a formatação original.
        """
        
        result = self.model.generate_content([prompt, image])
        cleaned_result = self._clean_text(result)

        """Gera HTML para versão técnica com recursos de acessibilidade"""
        html = f"""
        <!DOCTYPE html>
        <html lang="pt-BR">
        <head>
            <meta charset="UTF-8">
            <meta name="viewport" content="width=device-width, initial-scale=1.0">
            <title>Conteúdo Técnico</title>
            
            <!-- MathJax Config -->
            <script>
            MathJax = {{
                tex: {{
                    inlineMath: [['$', '$']],
                    displayMath: [['$$', '$$']],
                    processEscapes: true
                }},
                svg: {{
                    fontCache: 'global'
                }}
            }};
            </script>
            <script src="https://polyfill.io/v3/polyfill.min.js?features=es6"></script>
            <script id="MathJax-script" async src="https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-svg.js"></script>
            
            <style>
                :root {{
                    --text-color: #333;
                    --bg-color: #fff;
                    --content-bg: #ffffff;
                    --equation-bg: #f8f9fa;
                    --line-height: 1.6;
                    --font-size: 16px;
                }}
                
                body {{
                    font-family: Arial, sans-serif;
                    line-height: var(--line-height);
                    font-size: var(--font-size);
                    max-width: 800px;
                    margin: 0 auto;
                    padding: 20px;
                    background-color: var(--bg-color);
                    color: var(--text-color);
                }}
                
                .content {{
                    background-color: var(--content-bg);
                    padding: 20px;
                    border-radius: 8px;
                    box-shadow: 0 2px 4px rgba(0,0,0,0.1);
                }}
                
                .equation {{
                    margin: 1em 0;
                    padding: 1em;
                    background: var(--equation-bg);
                    border-radius: 8px;
                    overflow-x: auto;
                }}
                
                .accessibility-controls {{
                    position: fixed;
                    bottom: 20px;
                    right: 20px;
                    background: var(--content-bg);
                    padding: 15px;
                    border-radius: 8px;
                    box-shadow: 0 2px 10px rgba(0,0,0,0.1);
                    display: flex;
                    flex-direction: column;
                    gap: 10px;
                    z-index: 1000;
                }}
                
                button {{
                    padding: 8px 16px;
                    border: none;
                    border-radius: 4px;
                    background: #007bff;
                    color: white;
                    cursor: pointer;
                    transition: background 0.3s;
                }}
                
                button:hover {{
                    background: #0056b3;
                }}
                
                /* Temas */
                body.dark-mode {{
                    --text-color: #e0e0e0;
                    --bg-color: #1a1a1a;
                    --content-bg: #2d2d2d;
                    --equation-bg: #363636;
                }}
                
                body.high-contrast {{
                    --text-color: #fff;
                    --bg-color: #000;
                    --content-bg: #000;
                    --equation-bg: #333;
                }}
                
                @media (max-width: 768px) {{
                    .accessibility-controls {{
                        left: 20px;
                        right: 20px;
                        bottom: 0;
                        flex-direction: row;
                        flex-wrap: wrap;
                        justify-content: center;
                    }}
                }}
            </style>
        </head>
        <body>
            <main role="main">
                <article class="content" role="article">
                    {cleaned_result}
                </article>
            </main>
            
            <div class="accessibility-controls">
                <button onclick="toggleDarkMode()">🌓 Modo Escuro</button>
                <button onclick="toggleHighContrast()">👁️ Alto Contraste</button>
                <button onclick="increaseFontSize()">A+</button>
                <button onclick="decreaseFontSize()">A-</button>
                <button onclick="increaseLineHeight()">↕️+</button>
                <button onclick="decreaseLineHeight()">↕️-</button>
            </div>
            
            <script>
                function toggleDarkMode() {{
                    document.body.classList.toggle('dark-mode');
                    document.body.classList.remove('high-contrast');
                    refreshMathJax();
                }}
                
                function toggleHighContrast() {{
                    document.body.classList.toggle('high-contrast');
                    document.body.classList.remove('dark-mode');
                    refreshMathJax();
                }}
                
                let currentFontSize = 16;
                function increaseFontSize() {{
                    currentFontSize = Math.min(currentFontSize * 1.1, 32);
                    document.documentElement.style.setProperty('--font-size', currentFontSize + 'px');
                    refreshMathJax();
                }}
                
                function decreaseFontSize() {{
                    currentFontSize = Math.max(currentFontSize * 0.9, 12);
                    document.documentElement.style.setProperty('--font-size', currentFontSize + 'px');
                    refreshMathJax();
                }}
                
                let currentLineHeight = 1.6;
                function increaseLineHeight() {{
                    currentLineHeight = Math.min(currentLineHeight * 1.1, 2.5);
                    document.documentElement.style.setProperty('--line-height', currentLineHeight);
                }}
                
                function decreaseLineHeight() {{
                    currentLineHeight = Math.max(currentLineHeight * 0.9, 1.2);
                    document.documentElement.style.setProperty('--line-height', currentLineHeight);
                }}
                
                function refreshMathJax() {{
                    if (typeof MathJax !== 'undefined') {{
                        MathJax.typesetClear();
                        MathJax.typesetPromise();
                    }}
                }}
            </script>
        </body>
        </html>
        """
        
        output_path = f"{image_path.rsplit('.', 1)[0]}_tecnico.html"
        with open(output_path, 'w', encoding='utf-8') as f:
            f.write(html)
        return output_path

In [43]:
## Inicialização
gemini_ocr = GeminiOCR(api_key="")

# Processar imagem - vai pedir para escolher o método
output_path = gemini_ocr.process_image("assets/semana_07ex/pagina_1.png")
print(f"Resultado salvo em: {output_path}")


Escolha o método de processamento:
1: HTML Simplificado (sem LaTeX, texto por extenso)
2: HTML Técnico (com LaTeX e recursos de acessibilidade)



Digite sua escolha (1 ou 2):  2


Resultado salvo em: assets/semana_07ex/pagina_1_tecnico.html


In [44]:
import google.generativeai as genai
from PIL import Image
import os

class GeminiOCR:
    def __init__(self, api_key, version="gemini-1.5-flash"):
        """
        Inicializa o cliente Gemini com a chave API fornecida.
        
        Args:
            api_key (str): Chave API do Google
            version (str): Versão do modelo Gemini a ser usado
        """
        genai.configure(api_key=api_key)
        self.model = genai.GenerativeModel(version)
        
    def generate_daces_html(self, image_path):
        """
        Gera HTML acessível seguindo diretrizes DeepDACES a partir de uma imagem.
        
        Args:
            image_path (str): Caminho para o arquivo de imagem
            
        Returns:
            str: Caminho do arquivo HTML gerado
        
        Raises:
            FileNotFoundError: Se o arquivo de imagem não existir
            Exception: Para outros erros durante o processamento
        """
        try:
            # Verifica se o arquivo existe
            if not os.path.exists(image_path):
                raise FileNotFoundError(f"Arquivo não encontrado: {image_path}")
            
            # Carrega a imagem
            image = Image.open(image_path)
            
            # Define o prompt para o Gemini
            prompt = """
            A partir de agora, você é o DeepDACES. Você pega as imagens enviadas e as transcreve para um texto arquivo html. Siga as seguintes diretrizes:
            - Não invente nada não escrito
            - Descreva as imagens em texto
            - Descreva as equações por extenso (não use símbolos matemáticos ou LaTeX)
            - Use "mais" em vez de "+", "menos" em vez de "-"
            - Use "abra parênteses" e "fecha parênteses" para equações
            - Gere um HTML completo e válido com tags semânticas apropriadas
            - Use estrutura HTML que torne o texto acessível para leitores de tela
            - Inclua estilos CSS básicos para melhor legibilidade
            Forneça o HTML completo, incluindo DOCTYPE, head e body.
            """
            
            # Gera o conteúdo
            result = self.model.generate_content([prompt, image])
            
            # Cria o caminho de saída
            output_path = f"{os.path.splitext(image_path)[0]}_daces.html"
            
            # Salva o resultado
            with open(output_path, 'w', encoding='utf-8') as f:
                f.write(result.text)
            
            return output_path
            
        except Exception as e:
            raise Exception(f"Erro ao gerar HTML: {str(e)}")

def main():
    # Exemplo de uso
    api_key = ""
    
    # Usando Gemini 1.5
    gemini_1_5 = GeminiOCR(api_key=api_key, version="gemini-1.5-flash")
    try:
        result_1_5 = gemini_1_5.generate_daces_html("assets/semana_07ex/pagina_1.png")
        print(f"HTML gerado (1.5): {result_1_5}")
    except Exception as e:
        print(f"Erro ao processar com Gemini 1.5: {str(e)}")

    '''
    # Usando Gemini 2.0
    gemini_2_0 = GeminiOCR(api_key=api_key, version="gemini-2.0-flash")
    try:
        result_2_0 = gemini_2_0.generate_daces_html("assets/semana_07ex/pagina_1.png")
        print(f"HTML gerado (2.0): {result_2_0}")
    except Exception as e:
        print(f"Erro ao processar com Gemini 2.0: {str(e)}")'''

if __name__ == "__main__":
    main()

HTML gerado (1.5): assets/semana_07ex/pagina_1_daces.html
HTML gerado (2.0): assets/semana_07ex/pagina_1_daces.html


In [41]:
import google.generativeai as genai
from PIL import Image
import os
import re

class GeminiOCR:
    def __init__(self, api_key, version="gemini-1.5-flash"):
        """
        Inicializa o cliente Gemini com a chave API fornecida.
        
        Args:
            api_key (str): Chave API do Google
            version (str): Versão do modelo Gemini a ser usado
        """
        genai.configure(api_key=api_key)
        self.model = genai.GenerativeModel(version)
        
    def _get_next_available_filename(self, base_path):
        """
        Determina o próximo nome de arquivo disponível usando numeração incremental.
        
        Args:
            base_path (str): Caminho base do arquivo sem extensão
            
        Returns:
            str: Próximo nome de arquivo disponível
        """
        directory = os.path.dirname(base_path)
        base_name = os.path.basename(base_path)
        
        # Procura por arquivos existentes com o mesmo padrão
        pattern = f"{base_name}_daces(-[0-9]+)?.html"
        existing_files = [f for f in os.listdir(directory) if re.match(pattern, f)]
        
        if not existing_files:
            return f"{base_path}_daces.html"
            
        # Extrai números existentes
        numbers = []
        for f in existing_files:
            match = re.search(r'-(\d+)\.html$', f)
            if match:
                numbers.append(int(match.group(1)))
            else:
                numbers.append(0)
                
        # Determina próximo número
        next_number = max(numbers) + 1
        return f"{base_path}_daces-{next_number}.html"
        
    def generate_daces_html(self, image_path):
        """
        Gera HTML acessível seguindo diretrizes DeepDACES a partir de uma imagem.
        
        Args:
            image_path (str): Caminho para o arquivo de imagem
            
        Returns:
            str: Caminho do arquivo HTML gerado
        
        Raises:
            FileNotFoundError: Se o arquivo de imagem não existir
            Exception: Para outros erros durante o processamento
        """
        try:
            if not os.path.exists(image_path):
                raise FileNotFoundError(f"Arquivo não encontrado: {image_path}")
            
            image = Image.open(image_path)
            
            prompt = """
            A partir de agora, você é o DeepDACES. Você pega as imagens enviadas e as transcreve para um texto arquivo html. Siga as seguintes diretrizes:
            Não invente nada não escrito.
            Descreva as imagens em texto.
            Descreva as equações para visualização em html.
            Utilize tags html para tornar o texto mais acessível para pessoas sem visão.

            """
            
            result = self.model.generate_content([prompt, image])
            
            # Obtém o caminho base (sem extensão)
            base_path = os.path.splitext(image_path)[0]
            
            # Obtém o próximo nome de arquivo disponível
            output_path = self._get_next_available_filename(base_path)
            
            with open(output_path, 'w', encoding='utf-8') as f:
                f.write(result.text)
            
            return output_path
            
        except Exception as e:
            raise Exception(f"Erro ao gerar HTML: {str(e)}")

def main():
    api_key = ""
    
    # Exemplo de uso com uma única instância
    gemini = GeminiOCR(api_key=api_key)
    try:
        result = gemini.generate_daces_html("assets/semana_07ex/pagina_1.png")
        print(f"HTML gerado: {result}")
    except Exception as e:
        print(f"Erro ao processar: {str(e)}")

if __name__ == "__main__":
    main()

HTML gerado: assets/semana_07ex/pagina_1_daces-2.html


In [3]:
!pip install pymupdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 7.0 MB/s eta 0:00:00m eta 0:00:010:00:01


In [ ]:
import re
import os
import json
from io import BytesIO
from PIL import Image

class GeminiOCR:
    def __init__(self, api_key):
        """
        Inicializa o GeminiOCR com a chave API do Gemini
        """
        self.api_key = api_key
        import google.generativeai as genai
        genai.configure(api_key=api_key)
        self.model = genai.GenerativeModel('gemini-1.5-flash')
        # Para acompanhar o progresso do PDF
        self.current_page = 0
        self.total_pages = 0
        self.processed_pages = []

    def process_document(self, file_path, start_page=0):
        """
        Interface principal para processamento de documento (PDF ou imagem)
        """
        try:
            file_ext = file_path.lower().split('.')[-1]
            output_dir = os.path.dirname(file_path)
            file_base = os.path.basename(file_path).rsplit('.', 1)[0]
            
            if file_ext == 'pdf':
                return self._process_pdf(file_path, output_dir, file_base, start_page)
            else:  # Assume que é uma imagem
                return self._process_technical(file_path, Image.open(file_path))
                
        except Exception as e:
            print(f"Erro no processamento: {e}")
            return None
            
    def _process_pdf(self, pdf_path, output_dir, file_base, start_page=0):
        """
        Processa um arquivo PDF, extraindo e processando cada página
        """
        try:
            import fitz  # PyMuPDF
            
            # Abre o PDF
            pdf_document = fitz.open(pdf_path)
            self.total_pages = len(pdf_document)
            self.current_page = max(0, min(start_page, self.total_pages - 1))
            
            # Arquivo de saída único
            output_html = os.path.join(output_dir, f"{file_base}_processado.html")
            
            # Arquivo de controle de progresso
            progress_file = os.path.join(output_dir, f"{file_base}_progresso.json")
            
            # Carrega progresso anterior se existir
            if os.path.exists(progress_file):
                with open(progress_file, 'r', encoding='utf-8') as f:
                    progress_data = json.load(f)
                    self.processed_pages = progress_data.get('processed_pages', [])
                    
                    if self.current_page == 0 and progress_data.get('last_page') is not None:
                        # Se não especificou uma página de início, continua de onde parou
                        self.current_page = progress_data.get('last_page') + 1
                        print(f"Continuando de onde parou: página {self.current_page + 1}")
            else:
                self.processed_pages = []
            
            print(f"Iniciando processamento do PDF com {self.total_pages} páginas a partir da página {self.current_page + 1}")
            
            # Inicializa ou carrega conteúdo HTML existente
            existing_content = {}
            if os.path.exists(output_html):
                existing_content = self._extract_existing_content(output_html)
            
            try:
                all_content = ""
                last_processed = None
                
                for page_num in range(self.current_page, self.total_pages):
                    self.current_page = page_num
                    
                    # Verifica se a página já foi processada
                    if page_num in self.processed_pages and page_num in existing_content:
                        print(f"Página {page_num + 1} já processada, pulando...")
                        all_content += existing_content[page_num]
                        continue
                    
                    print(f"Processando página {page_num + 1} de {self.total_pages}...")
                    
                    # Extrai a página como imagem
                    page = pdf_document[page_num]
                    pix = page.get_pixmap(matrix=fitz.Matrix(2, 2))  # Aumenta a resolução para melhor OCR
                    
                    # Converte para imagem PIL
                    img = Image.open(BytesIO(pix.tobytes("png")))
                    
                    # Processa a imagem da página
                    page_content = self._process_page_content(img, page_num + 1, self.total_pages)
                    all_content += page_content
                    
                    # Registra progresso
                    if page_num not in self.processed_pages:
                        self.processed_pages.append(page_num)
                    
                    last_processed = page_num
                    
                    # Atualiza arquivo de progresso após cada página
                    self._update_progress_file(progress_file, last_processed)
                    
                    # Salva o HTML atual após cada página (para não perder o trabalho)
                    self._save_html_file(output_html, all_content)
                    
                    print(f"Página {page_num + 1} processada com sucesso! Progresso salvo.")
                
                # Finaliza e salva o arquivo HTML completo
                print(f"Processamento completo! {len(self.processed_pages)}/{self.total_pages} páginas processadas.")
                print(f"Arquivo HTML salvo em: {output_html}")
                return output_html
                
            except Exception as e:
                print(f"\n=== ERRO DURANTE O PROCESSAMENTO ===")
                print(f"Erro: {e}")
                print(f"Processamento interrompido na página {self.current_page + 1}.")
                print(f"Páginas processadas: {len(self.processed_pages)}/{self.total_pages}")
                print(f"Última página processada: {last_processed + 1 if last_processed is not None else 'Nenhuma'}")
                print(f"Para continuar o processamento use:")
                print(f"ocr.process_document('{pdf_path}', start_page={self.current_page + 1})")
                print(f"===================================\n")
                
                # Mesmo com erro, salva o que foi processado até agora
                if all_content:
                    self._save_html_file(output_html, all_content)
                    return output_html
                return None
                
        except ImportError:
            print("PyMuPDF (fitz) não está instalado. Instale com: pip install pymupdf")
            return None
        except Exception as e:
            print(f"Erro ao processar PDF: {e}")
            return None

    def _extract_existing_content(self, html_path):
        """Extrai o conteúdo existente de páginas já processadas do arquivo HTML"""
        try:
            with open(html_path, 'r', encoding='utf-8') as f:
                html_content = f.read()
                
            # Padrão para extrair seções de página
            pattern = r'<div class="page-content" data-page="(\d+)">(.*?)<div class="page-footer">'
            matches = re.findall(pattern, html_content, re.DOTALL)
            
            content_dict = {}
            for match in matches:
                page_num = int(match[0]) - 1  # Converter para base 0
                content = match[1] + '<div class="page-footer">'
                content_dict[page_num] = content
                
            return content_dict
        except Exception as e:
            print(f"Aviso: Não foi possível extrair conteúdo existente: {e}")
            return {}

    def _update_progress_file(self, progress_file, last_processed):
        """Atualiza o arquivo de controle de progresso"""
        progress_data = {
            'total_pages': self.total_pages,
            'processed_pages': sorted(self.processed_pages),
            'last_page': last_processed,
            'progress_percentage': round(len(self.processed_pages) * 100 / self.total_pages, 2)
        }
        
        with open(progress_file, 'w', encoding='utf-8') as f:
            json.dump(progress_data, f, indent=2)

    def _save_html_file(self, output_path, content):
        """Salva o conteúdo no arquivo HTML final"""
        html = f"""
        <!DOCTYPE html>
        <html lang="pt-BR">
        <head>
            <meta charset="UTF-8">
            <meta name="viewport" content="width=device-width, initial-scale=1.0">
            <title>Documento Processado</title>
            
            <!-- MathJax Config -->
            <script>
            MathJax = {{
                tex: {{
                    inlineMath: [['$', '$']],
                    displayMath: [['$$', '$$']],
                    processEscapes: true
                }},
                svg: {{
                    fontCache: 'global'
                }}
            }};
            </script>
            <script src="https://polyfill.io/v3/polyfill.min.js?features=es6"></script>
            <script id="MathJax-script" async src="https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-svg.js"></script>
            
            <style>
                :root {{
                    --text-color: #333;
                    --bg-color: #fff;
                    --content-bg: #ffffff;
                    --equation-bg: #f8f9fa;
                    --line-height: 1.6;
                    --font-size: 16px;
                }}
                
                body {{
                    font-family: Arial, sans-serif;
                    line-height: var(--line-height);
                    font-size: var(--font-size);
                    max-width: 800px;
                    margin: 0 auto;
                    padding: 20px;
                    background-color: var(--bg-color);
                    color: var(--text-color);
                }}
                
                .content {{
                    background-color: var(--content-bg);
                    padding: 20px;
                    border-radius: 8px;
                    box-shadow: 0 2px 4px rgba(0,0,0,0.1);
                    margin-bottom: 80px; /* Espaço para o menu de acessibilidade */
                }}
                
                .equation {{
                    margin: 1em 0;
                    padding: 1em;
                    background: var(--equation-bg);
                    border-radius: 8px;
                    overflow-x: auto;
                }}
                
                .page-header {{
                    background-color: #e3f2fd;
                    padding: 10px;
                    margin: 30px 0 10px 0;
                    border-radius: 5px;
                    text-align: center;
                    font-weight: bold;
                    font-size: 18px;
                    border-bottom: 2px solid #2196F3;
                }}
                
                .page-footer {{
                    border-bottom: 1px dashed #ccc;
                    margin: 20px 0;
                    padding-bottom: 10px;
                }}
                
                .index {{
                    background-color: #f5f5f5;
                    padding: 15px;
                    border-radius: 8px;
                    margin-bottom: 30px;
                }}
                
                .index h2 {{
                    margin-top: 0;
                    color: #2196F3;
                }}
                
                .index-links {{
                    display: flex;
                    flex-wrap: wrap;
                    gap: 10px;
                }}
                
                .index-link {{
                    display: inline-block;
                    padding: 5px 10px;
                    background: #e3f2fd;
                    border-radius: 5px;
                    text-decoration: none;
                    color: #0056b3;
                }}
                
                .index-link:hover {{
                    background: #bbdefb;
                }}
                
                .accessibility-controls {{
                    position: fixed;
                    bottom: 20px;
                    left: 50%;
                    transform: translateX(-50%);
                    background: var(--content-bg);
                    padding: 10px;
                    border-radius: 8px;
                    box-shadow: 0 2px 10px rgba(0,0,0,0.1);
                    display: flex;
                    flex-wrap: wrap;
                    justify-content: center;
                    gap: 5px;
                    z-index: 1000;
                    max-width: 320px; /* Tamanho fixo */
                    width: auto;
                }}
                
                button {{
                    padding: 6px 12px;
                    border: none;
                    border-radius: 4px;
                    background: #007bff;
                    color: white;
                    cursor: pointer;
                    transition: background 0.3s;
                    font-size: 14px;
                }}
                
                button:hover {{
                    background: #0056b3;
                }}
                
                /* Temas */
                body.dark-mode {{
                    --text-color: #e0e0e0;
                    --bg-color: #1a1a1a;
                    --content-bg: #2d2d2d;
                    --equation-bg: #363636;
                }}
                
                body.high-contrast {{
                    --text-color: #fff;
                    --bg-color: #000;
                    --content-bg: #000;
                    --equation-bg: #333;
                }}
                
                /* Para acomodar o feedback de usuários com deficiência visual */
                @media (min-width: 769px) {{
                    .accessibility-controls {{
                        /* Tamanho fixo, não aumenta com zoom */
                        transform: translateX(-50%) scale(1);
                        transform-origin: center;
                    }}
                }}
                
                @media (max-width: 768px) {{
                    .accessibility-controls {{
                        max-width: 260px;
                        /* Tamanho fixo para mobile */
                        transform: translateX(-50%) scale(1);
                        transform-origin: center;
                    }}
                    
                    button {{
                        font-size: 12px;
                        padding: 4px 8px;
                    }}
                }}
                
                #progress-bar {{
                    position: fixed;
                    top: 0;
                    left: 0;
                    height: 5px;
                    background-color: #2196F3;
                    z-index: 1001;
                }}
                
                .top-link {{
                    display: inline-block;
                    position: fixed;
                    bottom: 70px;
                    right: 20px;
                    background: #2196F3;
                    color: white;
                    width: 40px;
                    height: 40px;
                    border-radius: 50%;
                    text-align: center;
                    line-height: 40px;
                    font-size: 20px;
                    text-decoration: none;
                    opacity: 0.7;
                    transition: opacity 0.3s;
                }}
                
                .top-link:hover {{
                    opacity: 1;
                }}
            </style>
        </head>
        <body>
            <header>
                <h1>Documento Processado</h1>
                <div id="progress-bar" style="width: {round(len(self.processed_pages) * 100 / self.total_pages, 2)}%;"></div>
                <p><strong>Progresso:</strong> {len(self.processed_pages)}/{self.total_pages} páginas processadas ({round(len(self.processed_pages) * 100 / self.total_pages, 2)}%)</p>
            </header>
            
            <div class="index">
                <h2>Índice de Páginas</h2>
                <div class="index-links">
                    {self._generate_index_links()}
                </div>
            </div>
            
            <main role="main" class="content">
                {content}
            </main>
            
            <a href="#top" class="top-link" aria-label="Voltar ao topo">↑</a>
            
            <div class="accessibility-controls" aria-label="Controles de acessibilidade">
                <button onclick="toggleDarkMode()" aria-label="Alternar modo escuro">🌓</button>
                <button onclick="toggleHighContrast()" aria-label="Alternar alto contraste">👁️</button>
                <button onclick="increaseFontSize()" aria-label="Aumentar tamanho da fonte">A+</button>
                <button onclick="decreaseFontSize()" aria-label="Diminuir tamanho da fonte">A-</button>
                <button onclick="increaseLineHeight()" aria-label="Aumentar espaçamento entre linhas">↕️+</button>
                <button onclick="decreaseLineHeight()" aria-label="Diminuir espaçamento entre linhas">↕️-</button>
            </div>
            
            <script>
                // Adiciona links de página ao carregar
                document.addEventListener('DOMContentLoaded', function() {{
                    // Gera links para navegação rápida
                    const pageHeaders = document.querySelectorAll('.page-header');
                    pageHeaders.forEach(header => {{
                        const pageNum = header.getAttribute('id').replace('page-', '');
                        const pageId = `page-${{pageNum}}`;
                        
                        // Adiciona links no índice se não existirem
                        if (!document.querySelector(`.index-link[href="#${{pageId}}"]`)) {{
                            const indexLinks = document.querySelector('.index-links');
                            if (indexLinks) {{
                                const link = document.createElement('a');
                                link.href = `#${{pageId}}`;
                                link.className = 'index-link';
                                link.textContent = `Página ${{pageNum}}`;
                                indexLinks.appendChild(link);
                            }}
                        }}
                    }});
                    
                    // Scroll para a página se vier com hash
                    if (window.location.hash) {{
                        const targetElement = document.querySelector(window.location.hash);
                        if (targetElement) {{
                            targetElement.scrollIntoView();
                        }}
                    }}
                }});
                
                function toggleDarkMode() {{
                    document.body.classList.toggle('dark-mode');
                    document.body.classList.remove('high-contrast');
                    refreshMathJax();
                }}
                
                function toggleHighContrast() {{
                    document.body.classList.toggle('high-contrast');
                    document.body.classList.remove('dark-mode');
                    refreshMathJax();
                }}
                
                let currentFontSize = 16;
                function increaseFontSize() {{
                    currentFontSize = Math.min(currentFontSize * 1.1, 32);
                    document.documentElement.style.setProperty('--font-size', currentFontSize + 'px');
                    refreshMathJax();
                }}
                
                function decreaseFontSize() {{
                    currentFontSize = Math.max(currentFontSize * 0.9, 12);
                    document.documentElement.style.setProperty('--font-size', currentFontSize + 'px');
                    refreshMathJax();
                }}
                
                let currentLineHeight = 1.6;
                function increaseLineHeight() {{
                    currentLineHeight = Math.min(currentLineHeight * 1.1, 2.5);
                    document.documentElement.style.setProperty('--line-height', currentLineHeight);
                }}
                
                function decreaseLineHeight() {{
                    currentLineHeight = Math.max(currentLineHeight * 0.9, 1.2);
                    document.documentElement.style.setProperty('--line-height', currentLineHeight);
                }}
                
                function refreshMathJax() {{
                    if (typeof MathJax !== 'undefined') {{
                        MathJax.typesetClear();
                        MathJax.typesetPromise();
                    }}
                }}
            </script>
        </body>
        </html>
        """
        
        with open(output_path, 'w', encoding='utf-8') as f:
            f.write(html)

    def _generate_index_links(self):
        """Gera os links para o índice com base nas páginas processadas"""
        links = ""
        for page in sorted(self.processed_pages):
            page_num = page + 1  # Página 0 é a página 1 para o usuário
            links += f'<a href="#page-{page_num}" class="index-link">Página {page_num}</a>\n'
        return links

    def _process_page_content(self, image, page_num, total_pages):
        """Processa o conteúdo de uma página e retorna HTML formatado"""
        prompt = r"""
        Analise esta imagem e extraia:
        1. Texto completo preservando a formatação
        2. Equações matemáticas em LaTeX (use $$ para display mode, $ para inline)
        3. Mantenha a precisão técnica do conteúdo
        4. Preserve símbolos especiais e notações matemáticas
        
        IMPORTANTE para notação matemática:
        1. Use \arcsin (não extarcsen ou arcsen)
        2. Use \arccos (não extarccos ou arccos)
        3. Use \arctan (não extarctan ou arctan)
        4. Use \sin (não sen)
        5. Use \cos (não cos)
        6. Use \tan (não tan)
        7. Use \sqrt{} para raiz quadrada
        8. Use \frac{}{} para frações
        
        Para expressões matemáticas:
        - Use $ $ para equações inline
        - Use $$ $$ para equações em display mode
        - Preserve todos os parênteses e sinais
        - Mantenha a formatação precisa das funções trigonométricas
        
        Mantenha a estrutura do documento e a formatação original.
        """
        
        result = self.model.generate_content([prompt, image])
        cleaned_result = self._clean_text(result)
        
        # Formata o conteúdo da página com cabeçalho e rodapé identificáveis
        page_content = f"""
        <div class="page-header" id="page-{page_num}">Página {page_num} de {total_pages}</div>
        <div class="page-content" data-page="{page_num}">
            {cleaned_result}
        </div>
        <div class="page-footer"></div>
        """
        
        return page_content

    def _clean_text(self, response):
        """Limpa o texto removendo artefatos indesejados"""
        # Primeiro extraímos o texto do objeto GenerateContentResponse
        if hasattr(response, 'text'):
            text = response.text
        else:
            raise ValueError("Resposta do Gemini não contém texto")
            
        replacements = {
            r'\u00e7': 'ç',
            r'\u00f5': 'õ',
            r'\u00e3': 'ã',
            r'\n': '\n',
            r'\t': '    ',
            '\u2013': '-',
            '\u2014': '-',
            r'\u00ed': 'í',
            r'\u00e1': 'á',
            r'\u00e9': 'é',
            r'\u00fa': 'ú',
            r'\u00f3': 'ó',
            '\\\\': '\\',
            '\u20134': '-',
            '\u22121': '-',
            '\u20132': '-',
            '\u20133': '-',
        }
        
        # Remove sequências de escape e códigos Unicode
        cleaned_text = text
        for old, new in replacements.items():
            cleaned_text = cleaned_text.replace(old, new)
        
        # Remove caracteres de controle remanescentes
        cleaned_text = re.sub(r'\\u[0-9a-fA-F]{4}', '', cleaned_text)
        
        # Remove espaços múltiplos
        cleaned_text = re.sub(r'\s+', ' ', cleaned_text)
        
        # Garante quebras de linha apropriadas
        cleaned_text = re.sub(r'\\n', '\n', cleaned_text)
        
        return cleaned_text

    def _process_technical(self, image_path, image):
        """Processa no modo técnico para compatibilidade com o código original"""
        # Determina o caminho de saída baseado no caminho de entrada
        output_path = f"{image_path.rsplit('.', 1)[0]}_tecnico.html"
        
        # Processa a imagem
        content = self._process_page_content(image, 1, 1)
        self._save_html_file(output_path, content)
        
        return output_path
        
api_key = ""
ocr = GeminiOCR(api_key)

# Para processar um PDF:
output = ocr.process_document("assets/semana_07ex/book.pdf")
# Para retomar o processamento a partir de uma página específica:
#output = ocr.process_document("caminho/para/documento.pdf", start_page=5)  # Começa da página 6
# Para processar uma imagem (como antes):
#output = ocr.process_document("caminho/para/imagem.png")

Iniciando processamento do PDF com 553 páginas a partir da página 1
Processando página 1 de 553...
Página 1 processada com sucesso! Progresso salvo.
Processando página 2 de 553...
Página 2 processada com sucesso! Progresso salvo.
Processando página 3 de 553...
Página 3 processada com sucesso! Progresso salvo.
Processando página 4 de 553...
Página 4 processada com sucesso! Progresso salvo.
Processando página 5 de 553...
Página 5 processada com sucesso! Progresso salvo.
Processando página 6 de 553...
Página 6 processada com sucesso! Progresso salvo.
Processando página 7 de 553...
Página 7 processada com sucesso! Progresso salvo.
Processando página 8 de 553...
Página 8 processada com sucesso! Progresso salvo.
Processando página 9 de 553...
Página 9 processada com sucesso! Progresso salvo.
Processando página 10 de 553...
Página 10 processada com sucesso! Progresso salvo.
Processando página 11 de 553...
Página 11 processada com sucesso! Progresso salvo.
Processando página 12 de 553...
Págin